In [1]:
import pandas as pd
import nltk
from tqdm import tqdm
from nltk import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
#from Levenshtein import distance
import psycopg2
import random
import json

In [ ]:
nltk.download('wordnet')

In [ ]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [2]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [3]:
from string import punctuation
punctuation_small_set = punctuation.replace("-","")
punctuation_small_set = punctuation_small_set.replace("'","")
punctuation_small_set

'!"#$%&()*+,./:;<=>?@[\\]^_`{|}~'

In [ ]:
nltk.word_tokenize("sister-in-law  is here")

In [ ]:
w = 'мама'
p = morph.parse(w)[0]
lemma = p.normal_form
pos = p.tag.POS
lemma

In [ ]:
nltk.pos_tag(nltk.word_tokenize("sister-in-law  is here"))

In [ ]:
def get_word_pos_lemma_list(clean_ngramm):
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    word_pos_list = nltk.pos_tag(tkn)
    for word_pos in word_pos_list:
        wordnet_pos = get_wordnet_pos(word_pos[1]) 
        if (wordnet_pos):
            lemma = lemmatizer.lemmatize(word_pos[0], pos = wordnet_pos)
        else:
            lemma = lemmatizer.lemmatize(word_pos[0])
        element = word_pos[0] + "_" + word_pos[1] + "_" + lemma
        final_ngramm.append(element)
    return final_ngramm

In [4]:
def det_lemmatized_popular_ngramm(popular_ngramm, popular_ngramm_ids,popular_ngramm_ref_ids,popular_ngramm_set_ids, debug = False):
    ngramm_list = []
    for word_ind in tqdm(range(len(popular_ngramm))):
        word_id = popular_ngramm_ids[word_ind]
        ngramm = popular_ngramm[word_ind]
        ref_id = popular_ngramm_ref_ids[word_ind]
        set_id = popular_ngramm_set_ids[word_ind]
        clean_ngramm = ''
        for char in ngramm:
            if char not in punctuation: #punctuation_small_set
                clean_ngramm += char
            else:
                clean_ngramm += ' '
        if debug: print(clean_ngramm)
        final_ngramm = get_word_pos_lemma_id_list(clean_ngramm,word_id,ref_id,set_id)
        ngramm_list.append(final_ngramm)
        """
        try:
            final_ngramm = get_word_pos_lemma_id_list(clean_ngramm,word_id)
            ngramm_list.append(final_ngramm)
            if debug:print(final_ngramm)
        except:
            if debug:print("FAILED",ngramm)
            pass
        """
    return ngramm_list

In [ ]:
def get_word_pos_lemma_id_list(clean_ngramm,word_id):
    

In [5]:
def get_word_pos_lemma_id_list(clean_ngramm,word_id, ref_id, set_id):
    final_ngramm = []
    tkn = nltk.word_tokenize(clean_ngramm)
    #print(tkn)
    for word in tkn:
        p = morph.parse(word)[0]
        lemma = p.normal_form
        pos = p.tag.POS
        #print(lemma,pos)
        if not pos:
            pos = "nodetected"
        element = word + "_" + pos + "_" + lemma + '_' + str(word_id) + '_' + str(ref_id) + '_' + str(set_id)
        final_ngramm.append(element)
    return final_ngramm
get_word_pos_lemma_id_list("мама мыла", 10,1,1)

['мама_NOUN_мама_10_1_1', 'мыла_NOUN_мыло_10_1_1']

In [6]:
def get_alph_dict(ngramms_pos_lemm_list):
    alph_dict = {}
    for ngramm in ngramms_pos_lemm_list:
        for elemetn in ngramm:
            word = elemetn.split("_")[0]
            first_two_el = min(2,len(word))
            first_two_letters = word[:first_two_el]
            if first_two_letters in alph_dict:
                alph_dict[first_two_letters].append(ngramm)
            else:
                alph_dict[first_two_letters] = []
                alph_dict[first_two_letters].append(ngramm)
    return alph_dict
#bigr_dict = get_alph_dict(bigramm)


In [ ]:
b_db = pd.read_csv("top_10k_bigr.csv")
popular_bigramms = list(b_db['translate'])
popular_bigramms_id = list(b_db['word_id'])
popular_bigramms_ref_id = list(b_db['ref_id'])
popular_bigramms_set_id = list(b_db['setting_id'])
b_db.head()

In [ ]:
bigramm = det_lemmatized_popular_ngramm(popular_bigramms,popular_bigramms_id,
                                        popular_bigramms_ref_id,popular_bigramms_set_id, debug = False)

In [ ]:
bigr_dict = get_alph_dict(bigramm)

In [ ]:
bigramm[0]

In [ ]:
with open("bigramms_top.json" ,"w") as f:
    json.dump(bigramm, f, indent = 4, ensure_ascii=False)
with open("bigramms_alpha_dict.json" ,"w") as f:
    json.dump(bigr_dict, f, indent = 4, ensure_ascii=False)

In [ ]:
popular_trigramms = pd.read_csv("top_10k_trigr.csv")
popular_trigramms_list = list(popular_trigramms['translate'])
popular_trigramms_list_id = list(popular_trigramms['word_id'])
popular_trigramms_ref_id = list(popular_trigramms['ref_id'])
popular_trigramms_set_id = list(popular_trigramms['setting_id'])
popular_trigramms.head()

In [ ]:
trigramm = det_lemmatized_popular_ngramm(popular_trigramms_list,popular_trigramms_list_id,
                                        popular_trigramms_ref_id,
                                        popular_trigramms_set_id)
trigramm_dict = get_alph_dict(trigramm)
with open("trigramms_top.json" ,"w") as f:
    json.dump(trigramm, f, indent = 4, ensure_ascii=False)
with open("trigramms_dict.json" ,"w") as f:
    json.dump(trigramm_dict, f, indent = 4, ensure_ascii=False)

In [7]:
def get_top_prop(db):
    popular_qgramms = pd.read_csv(db)
    popular_qgramms_list = list(popular_qgramms['translate'])
    ppopular_qgramms_list_id = list(popular_qgramms['word_id'])
    popular_qgramms_ref_id = list(popular_qgramms['ref_id'])
    popular_qgramms_set_id = list(popular_qgramms['setting_id'])
    print(popular_qgramms.head())
    return popular_qgramms_list, ppopular_qgramms_list_id, popular_qgramms_ref_id, popular_qgramms_set_id

In [8]:
popular_qgramms_list, ppopular_qgramms_list_id, popular_qgramms_ref_id, popular_qgramms_set_id = get_top_prop("top_10k_qgr.csv")

   word_id  ref_id  setting_id                                  translate
0    37065       1           1                       родной брат и сестра
1     3472       1           1                   рука (от кисти до плеча)
2    12881       1           1  испытывающий / чувствующий головокружение
3  5548373       1           1        с наибольшим количеством просмотров
4     5453       1           1           блок, кубик, многоквартирный дом


In [10]:
popular_qgramms_list[:10]

['родной брат и сестра',
 'рука (от кисти до плеча)',
 'испытывающий / чувствующий головокружение',
 'с наибольшим количеством просмотров',
 'блок, кубик, многоквартирный дом',
 'перерыв, перемена (в школе), перелом',
 'общий, всеобщий, публичный, распространённый',
 'указывает на направление к, в, на',
 'ещё один, другой, новый',
 'останавливаться (на короткое время)']

In [12]:
def save_reference(prefix, word_list, id_list, ref_id_list, set_id_list):
    ngramm = det_lemmatized_popular_ngramm(word_list,id_list,ref_id_list,set_id_list)
    ngramm_dict = get_alph_dict(ngramm)
    with open(prefix +"gramms_top.json" ,"w") as f:
        json.dump(ngramm, f, indent = 4, ensure_ascii=False)
    with open(prefix + "gramms_dict.json" ,"w") as f:
        json.dump(ngramm_dict, f, indent = 4, ensure_ascii=False)

In [13]:
save_reference("q", popular_qgramms_list, ppopular_qgramms_list_id, popular_qgramms_ref_id, popular_qgramms_set_id)

100%|██████████| 10000/10000 [00:18<00:00, 551.89it/s]


In [ ]:
popular_qgramms = pd.read_csv("top_rating_qgramms.csv")
popular_qgramms_list = list(popular_qgramms['word_value'])
popular_qgramms_list_id = list(popular_qgramms['word_id'])
popular_qgramms.head()

In [ ]:
qgramm = det_lemmatized_popular_ngramm(popular_qgramms_list,popular_qgramms_list_id)

In [ ]:
qgramm[0]

In [ ]:
qgramm_dict = get_alph_dict(qgramm)

In [ ]:
with open("qgramms_top.json" ,"w") as f:
    json.dump(qgramm, f, indent = 4)
with open("qgramms_dict.json" ,"w") as f:
    json.dump(qgramm_dict, f, indent = 4)

# LOOK FOR SIMILAR

In [ ]:
def get_orig_string (word_pos_list):
    orig_string = ''
    for el in word_pos_list:
        word = el.split("_")[0]
        orig_string += word + ' '
    orig_string = orig_string.strip()
    return orig_string


def get_lemmas_list (word_pos_list):
    lemma_list = []
    for el in word_pos_list:
        lemma = el.split("_")[2]
        lemma_list.append(lemma)
    return lemma_list
def get_pos_string (word_pos_list):
    pos_string = ''
    for el in word_pos_list:
        pos = el.split("_")[1]
        pos_string += pos
    return pos_string

def copare_lemmas_lists(compared_lemmas, lemmas_from_list, search_range, debug = False):
    for lemma_comp in compared_lemmas:
        for lemma_list in lemmas_from_list:
            if lemma_comp == lemma_list:
                if debug:print("SAME WORD")
                return False
            elif distance(lemma_comp, lemma_list) > 0 and distance(lemma_comp, lemma_list) <= search_range:
                continue
            else:
                if debug:print("TOO MUCH lemmas DIFFERENCE")
                return False
    return  True

def compare_word_pos_lemma_list(compared_wpl, from_list_wpl, search_range, debug = False):
    if debug: print("COMPARED",compared_wpl, "FROM LIST",from_list_wpl, "SEARCH RANGE",search_range)
    compared_lemma_list = get_lemmas_list(compared_wpl)
    from_list_lemma_list = get_lemmas_list(from_list_wpl)
    
    compared_pos_string = get_pos_string(compared_wpl)
    from_list_pos_string = get_pos_string(from_list_wpl)
    if debug:print(compared_lemma_list, from_list_lemma_list, compared_pos_string,from_list_pos_string)
    pos_check = False
    if distance(compared_pos_string, from_list_pos_string) > 0 and distance(compared_pos_string, from_list_pos_string) <= search_range:
        if debug: print("POS OK")
        pos_check = True
    else:
        if debug: print("TOO MUCH POS DIFFERENCE")
    lemmas_check = copare_lemmas_lists(compared_lemma_list, from_list_lemma_list, search_range)
    
    if pos_check == True and lemmas_check == True:
        if debug: print("all checks OK")
        return True
    

In [ ]:
def look_for_similar (offset, interval, similar_ngramms_list, debug = False):
    output_unigramm_json = []
    request = """SELECT word_id, word_lemma, word_rating, jdesc->>'wd' AS word_value
    FROM public.content_words WHERE(word_hash != 0) AND (((array_length(regexp_split_to_array(content_words.jdesc->>'wd', '[ ''-]'), 1) = 2)))
    ORDER BY word_rating DESC LIMIT """ + str(interval) + " OFFSET " + str(offset)
    print(request)
    conn.rollback()
    cursor = conn.cursor()
    cursor.execute(request)
    for row in cursor:
        curr_json = {}
        word_id = row[0]
        ngramm = row[3]
        print(ngramm)
        
        clean_ngramm = ''
        for char in ngramm:
            if char not in punctuation_small_set:
                clean_ngramm += char
            else:
                clean_ngramm += ' '
        clean_ngramm = clean_ngramm.strip()
        random.shuffle(similar_ngramms_list)
        word_pos_lemma_list = get_word_pos_lemma_list(clean_ngramm)
        collected_similar_ngramms = []
        collected_similar_ngramms_count = 0
        search_range = 2
        handled_similar_indexes = []
        while collected_similar_ngramms_count < 30 and search_range < 10:
            for similar_ngramm_index in range(len(similar_ngramms_list)):
                if (similar_ngramm_index not in handled_similar_indexes):
                    similar_ngramm = similar_ngramms_list[similar_ngramm_index]
                    comp_result = compare_word_pos_lemma_list(word_pos_lemma_list, similar_ngramm, search_range)
                    if comp_result == True:
                        handled_similar_indexes.append(similar_ngramm_index)
                        orig_line = get_orig_string(similar_ngramm)
                        collected_similar_ngramms.append(orig_line)
                        collected_similar_ngramms_count += 1
                        if debug: print("SIMILAR COLLECTED",collected_similar_ngramms_count )
                        if collected_similar_ngramms_count >= 30:
                            break
                else:
                    if debug: print("INDEX ALREADY HANDLED")
            search_range += 1
        print(collected_similar_ngramms)
        print("========")
        #curr_json = {"word_id":word_id, "word":word, "simlar_words": sim_words_dict[word]}
        #output_unigramm_json.append(curr_json)
        
    #write_response(output_unigramm_json,offset, offset + interval)

look_for_similar(0, 10, bigramm)

In [ ]:
bigramm[126]